In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/AI_Platform_(Unified)_SDK_Custom_Container_Prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial walks through simulating multimodel endpoint by building a custom container to serve a scikit-learn random forest and decision tree models on Vertex  same endpoint. You will use the FastAPI Python web server framework to create a prediction and health endpoint.
You will also cover incorporating a pre-processor from training into your online serving.


### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

This tutorial uses [the copy of the Iris dataset included in the
scikit-learn library](https://scikit-learn.org/stable/datasets/index.html#iris-dataset).

### Objective

The goal is to:
- Train a two separate models that uses a flower's measurements as input to predict what type of iris it is.
- Save the models and its serialized pre-processor
- Build a FastAPI server to handle predictions and health checks
- Build a custom container with models artifacts
- Upload and deploy custom container to Vertex Prediction
- Invoke the endpoint with two different payload to indicate which model to use
This tutorial focuses more on deploying multiple models with Vertex AI than on
the design of the models themselves.


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* Docker
* Git
* Google Cloud SDK (gcloud)
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as NumPy, Scikit-learn, FastAPI, Uvicorn, and joblib. Use the latest major GA version of each package.

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev

In [ ]:
# Required in Docker serving container
%pip install -U --user -r requirements.txt

# For local FastAPI development and running
%pip install -U --user "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63

# Vertex SDK for Python
%pip install -U --user google-cloud-aiplatform

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` or `%` as shell commands, and it interpolates Python variables with `$` or `{}` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

print("Project ID:", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebooks, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING") and not os.getenv(
        "GOOGLE_APPLICATION_CREDENTIALS"
    ):
        %env GOOGLE_APPLICATION_CREDENTIALS 

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS '<Path to service account Json file>'

### Configure project and resource names

In [ ]:
REGION = "us-central1"  # @param {type:"string"}
MODEL_ARTIFACT_DIR = "custom-container-prediction-model"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction"  # @param {type:"string"}
IMAGE = "sklearn-fastapi-server"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-custom-container"  # @param {type:"string"}

`REGION` - Used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that will be pushed.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI Model resource.

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To update your model artifacts without re-building the container, you must upload your model
artifacts and any custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

## Write your pre-processor
Scaling training data so each numerical feature column has a mean of 0 and a standard deviation of 1 [can improve your model](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data).

Create `preprocess.py`, which contains a class to do this scaling:

In [ ]:
%mkdir app

In [ ]:
%%writefile app/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds


## Train and store model with pre-processor
Next, use `preprocess.MySimpleScaler` to preprocess the iris data, then train a model using scikit-learn.

At the end, export your trained model as a joblib (`.joblib`) file and export your `MySimpleScaler` instance as a pickle (`.pkl`) file:

In [ ]:
%cd app/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X, y)

# Decision Tree(using less features than Random forrest)

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X[:,:-1], y)


joblib.dump(random_forest_model, "random_forest_model.joblib")
joblib.dump(decision_tree_model, "decision_tree_model.joblib")

with open("preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [ ]:
!gsutil cp random_forest_model.joblib decision_tree_model.joblib preprocessor.pkl {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/
%cd ..

In [ ]:
! echo {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}

## Build a FastAPI server

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris


app = FastAPI()
gcs_client = storage.Client()

# downloading multiple models

with open("preprocessor.pkl", 'wb') as preprocessor_f, open("random_forest_model.joblib", 'wb') as random_forest_model_f, open("decision_tree_model.joblib", 'wb') as decision_tree_model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/preprocessor.pkl", preprocessor_f
    )
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/random_forest_model.joblib", random_forest_model_f
    )
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/decision_tree_model.joblib", decision_tree_model_f
    )

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

_class_names = load_iris().target_names
_random_forest_model = joblib.load("random_forest_model.joblib")
_decision_tree_model = joblib.load("decision_tree_model.joblib")
_preprocessor = preprocessor


@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    parameters = body["parameters"] # catch the parameter from the body payload
    instances = body["instances"]
    if(parameters["model_name"]=="random_forest"): # use the model name to decide which model to invoke
        print(parameters["model_name"])
        inputs = np.asarray(instances)
        preprocessed_inputs = _preprocessor.preprocess(inputs)
        outputs = _random_forest_model.predict(preprocessed_inputs)
    else:
        print(parameters["model_name"])
        inputs = np.asarray(instances)
        outputs = _decision_tree_model.predict(inputs)

    return {"predictions": [_class_names[class_num] for class_num in outputs]}


### Add pre-start script
FastAPI will execute this script before starting up the server. The `PORT` environment variable is set to equal `AIP_HTTP_PORT` in order to run FastAPI on same the port expected by Vertex AI.

In [ ]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

### Store test instances to use later
To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
## to invoke random forest use this

In [ ]:
%%writefile instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ],
    "parameters":{"model_name":"random_forest"}
}

In [ ]:
## to invoke decision tree  use this

In [ ]:
%%writefile instances2.json
{
    "instances": [
        [6.7, 3.1, 4.7],
        [4.6, 3.1, 1.5]
    ],
    "parameters":{"model_name":"decision_tree"}
}

## Build and push container to Artifact Registry

### Build your container
Optionally copy in your credentials to run the container locally.

In [ ]:
# NOTE: Copy in credentials to run locally, this step can be skipped for deployment
%cp $GOOGLE_APPLICATION_CREDENTIALS app/credentials.json

Write the Dockerfile, using `tiangolo/uvicorn-gunicorn-fastapi` as a base image. This will automatically run FastAPI for you using Gunicorn and Uvicorn. Visit [the FastAPI docs to read more about deploying FastAPI with Docker](https://fastapi.tiangolo.com/deployment/docker/).

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

Build the image and tag the Artifact Registry path that you will push to.

In [ ]:
!sudo docker build \
    --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE} \
    .

### Run and test the container locally (optional)

Run the container locally in detached mode and provide the environment variables that the container requires. These env vars will be provided to the container by Vertex Prediction once deployed. Test the `/health` and `/predict` routes, then stop the running image.

In [ ]:
!sudo docker rm local-iris

In [ ]:
#Run this a a new terminal to deploy the endpoint locally,then open a new terminal and use it as a client
#sudo docker run  -p 80:9999 -e AIP_HTTP_PORT=9090  -e AIP_HTTP_PORT=9090  --name=local-iris -e AIP_HTTP_PORT=9090 -e AIP_HEALTH_ROUTE=/health -e AIP_PREDICT_ROUTE=/predict -e AIP_STORAGE_URI=gs://{BUCKET_NAME}/{MODEL_ARTIFACT_DIR} -e GOOGLE_APPLICATION_CREDENTIALS=credentials.json   {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

In [ ]:
!sudo docker run -d  -p 80:8080 \
    --name=local-iris \
    -e AIP_HTTP_PORT=8080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI={BUCKET_NAME}/{MODEL_ARTIFACT_DIR} \
    -e GOOGLE_APPLICATION_CREDENTIALS=credentials.json \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

In [ ]:
!sudo curl -X POST \
  -d @instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

In [ ]:
!sudo curl -X POST \
  -d @instances2.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

In [ ]:
!sudo docker stop local-iris

### Push the container to artifact registry

Configure Docker to access Artifact Registry. Then push your container image to your Artifact Registry repository.

In [ ]:
!gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION



In [ ]:
!sudo gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

In [ ]:
!gcloud auth list --filter=status:ACTIVE --format="value(account)"

In [ ]:
!sudo docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

## Deploy to Vertex AI

Use the Python SDK to upload and deploy your model.

### Upload the custom container model

In [ ]:
from google.cloud import aiplatform

In [ ]:
# brough syntanx from here : https://googleapis.dev/python/google-api-core/latest/auth.html#service-accounts

from google.oauth2 import service_account

cred = service_account.Credentials.from_service_account_file(
    '<PATH to service account json key>')

# alternativly, you can use compute engine credentials

from google.auth import compute_engine
cred = compute_engine.Credentials()

print(cred)


In [ ]:
aiplatform.init(project="hamzarhibi", location="us-central1",credentials=cred)


In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    credentials=cred,
    encryption_spec_key_name="<Path to your encryption key>" # make sure the service account used have permission to encrypt/decrypt using it
)

In [ ]:
print(f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}")

### Deploy the model on Vertex AI
After this step completes, the model is deployed and ready for online prediction.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4",encryption_spec_key_name="<Path to your encryption key>")

## Send predictions



### Using REST

In [ ]:

ENDPOINT_ID = "<endpoint ID>" # you can get this from the console

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### Using gcloud CLI

In [ ]:
!gcloud beta ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances2.json

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}